In [2]:
import torch
import polars as pl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

In [ ]:
df = pl.scan_parquet("data/out/internet-movie-script.parquet").with_row_index()
embeddings = torch.load("data/out/scripts-embedded.pt", weights_only=True)
print(df.shape)
print(embeddings.shape)
print(embeddings.dtype)

statistic,sr.no,movie_title_alphabetical,writers,genre,date_original,script_original,comma_count,script_length,fraction_commas,script,date,movie_title
str,f64,str,str,str,str,str,f64,f64,f64,str,str,str
"""count""",59.0,"""59""","""59""","""59""","""52""","""59""",59.0,59.0,59.0,"""59""","""52""","""59"""
"""null_count""",0.0,"""0""","""0""","""0""","""7""","""0""",0.0,0.0,0.0,"""0""","""7""","""0"""
"""mean""",30.525424,null,null,null,null,null,770.067797,33212.762712,0.025504,null,"""2003-08-03 07:50:46.153000""",null
"""std""",17.61304,null,null,null,null,null,1393.423381,1989.515262,0.04755,null,null,null
"""min""",1.0,"""15 Minutes""",""" Aaron Sorkin""","""Action""",""" January_1991""",""" FADE IN: …",71.0,26466.0,0.002115,""" FADE IN: …","""1964-12-01""","""15 Minutes"""
"""25%""",16.0,null,null,null,null,null,146.0,33596.0,0.004271,null,"""1998-12-01""",null
"""50%""",31.0,null,null,null,null,null,193.0,33705.0,0.005704,null,"""2007-05-01""",null
"""75%""",46.0,null,null,null,null,null,301.0,33942.0,0.008563,null,"""2010-11-01""",null
"""max""",60.0,"""Zootopia""","""William Steig,Ted Elliott""","""Family,Comedy,Musical,Fantasy,…","""September_2014""","""[Chinese guard is seen walking…",5014.0,37233.0,0.159554,"""[Chinese guard is seen walking…","""2019-04-01""","""Zootopia"""


torch.Size([59, 1024])
torch.float32


In [87]:
square_copied_over_0 = embeddings.unsqueeze(1).expand(-1, embeddings.shape[0], -1)
square_copied_over_1 = embeddings.unsqueeze(0).expand(embeddings.shape[0], -1, -1)
print(square_copied_over_0.shape)
print(square_copied_over_1.shape)

print(
    (square_copied_over_0[1, 13, :] == embeddings[1, :]).all()
)

print(
    (square_copied_over_1[13, 1, :] == embeddings[1, :]).all()
)

torch.Size([59, 59, 1024])
torch.Size([59, 59, 1024])
tensor(True)
tensor(True)


In [94]:
cos_sim_layer = torch.nn.CosineSimilarity(dim=2)

cos_sim = cos_sim_layer(square_copied_over_0, square_copied_over_1)

print(cos_sim.shape)

dotprod = torch.tensordot(embeddings, embeddings, dims=([1], [1]))

print(dotprod.shape)

torch.Size([59, 59])
torch.Size([59, 59])


In [108]:
np_cos_sim = cos_sim.flatten().numpy()
np_dotprod = dotprod.flatten().numpy()

csdf = pl.DataFrame(np_cos_sim)
dtdf = pl.DataFrame(np_dotprod)

csdf.columns = ["cos"]
dtdf.columns = ["dot"]

simdf = csdf.with_row_index().join(
    dtdf.with_row_index(),
    on=pl.col("index"),
    how="inner"
)
print(simdf.shape)
simdf.select(pl.col("cos", "dot")).corr()


(3481, 3)


cos,dot
f64,f64
1.0,0.68423
0.68423,1.0


In [141]:
def rank_similarity(
    movie_title: str,
    rank_function: str | pl.Expr,
    print_similarity_correlation: bool = True
) -> pl.DataFrame:
    
    idx = get_index_in_df_from_title(movie_title)
    

    simdf = pl.DataFrame(
        {"dotprod": dotprod[idx].numpy(), "cos_sim": cos_sim[idx].numpy()}
    ).with_row_index()\
        .join(
            df.select(pl.col("movie_title")).with_row_index().collect(),
            on = pl.col("index"),
            how = "inner"
        )
    if print_similarity_correlation is True:
        print_similarity_correlation_helper(simdf)

    return simdf.sort(rank_function, descending=True)

def get_index_in_df_from_title(movie_title: str) -> int:
    idx = df.filter(pl.col("movie_title").str.to_lowercase() == movie_title.lower()).select(pl.col("index")).collect()
    if len(idx) < 1:
        raise ValueError(f"Could not find movie {movie_title}")
    elif len(idx) > 1:
        raise ValueError(f"Found multiple matches for movie {movie_title}\n{idx}")
    else:
        idx = idx[0, "index"]
        return idx
    
def print_similarity_correlation_helper(
    data: pl.DataFrame,
    col1: str = "dotprod",
    col2: str = "cos_sim"
) -> None:
    corr_dotprod_cos = data.select(pl.col(col1, col2)).corr()\
            [0, col2]
    msg = "Correlation between dot product and cosine similarity "
    msg += f"across all movies: {corr_dotprod_cos:.3f}"
    print(msg)
    

In [142]:
rank_similarity("aladdin", "cos_sim")

Correlation between dot product and cosine similarity across all movies: 0.838


index,dotprod,cos_sim,movie_title
u32,f32,f32,str
2,226.992661,1.0,"""Aladdin"""
32,197.151505,0.865582,"""Shrek"""
22,191.269958,0.843673,"""The Little Mermaid"""
26,190.806,0.840338,"""Mulan"""
23,182.342819,0.828296,"""Mary Poppins"""
…,…,…,…
37,164.426224,0.750677,"""Transformers: The Movie"""
56,167.198135,0.745395,"""The Bourne Ultimatum"""
39,167.321442,0.740435,"""Wall-E"""
